# Exercise 1

The goal of this exercise is to model a function of the form
$$ z = f \left( (x - a)^2 + (y - b)^2 \right), $$ 
with a Neural Network (NN). Practically, you are searching for a mapping from $\mathbb{R}^2 \rightarrow \mathbb{R}$. Unfortunately, in the general case, you don't know the constant $a,b$ and the function $f$. However, you have a collection of data points $(x,y,z)$. 

1. Start by implementing a simple NN with linear layers that does not take into account of the function class. How hard is it to fit the data? 

2. Find a way to encode the data characteristics in the model. Is it still hard to fit the data? You can start by assuming that you know a and b.

3. Study the effect of the sample. Decrease the amount of data in the training set. What do you observe?

4. What about the case where you don't know a and b?


In [ ]:
# Autoreload is to always reload the imported python files.
%load_ext autoreload
%autoreload 2
# Matplotlib inline allows to make plot inline with the notebook with Matplotlib
%matplotlib inline

In [ ]:
# import all packages
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
import torch
import matplotlib.pyplot as plt
from torchsummary import summary
from poutyne import Model, SKLearnMetrics
from sklearn.metrics import r2_score, median_absolute_error
from utils import metric_flatten, get_poutyne_callbacks, saferm



In [ ]:
# These line allows to select the first GPU of the machine or the CPU is not GPU is present
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

## Dataset generation

In [ ]:
# Assume you don't know the data generating function.
# Later on assume you don't know a and b and try to recover them.
a = 0.2 
b = 0.8
def func(xx,yy):
    y = 2*np.cos( 3* ((xx-a)**2 + (yy-b)**2) )
    return y


In [ ]:
lim = 1.5
x = np.arange(-lim, lim, 0.03)
y = np.arange(-lim, lim, 0.03)
xx, yy = np.meshgrid(x, y, sparse=False)
z = func(xx,yy)
h = plt.contourf(x, y, z)
plt.colorbar()
plt.axis('scaled');


In [ ]:
## dataset creation
def sample_generator_regression(n):
    # RRandom sampling of points
    x = np.random.rand(n, 2).astype(np.float32)*2*lim -lim
    # compute the output
    # we expand the dimension to have the size [n x 1], which will be the output shape of the NN.
    y = np.expand_dims(func(x[:,0], x[:,1]),axis=1)
    # Convertion to Pytorch tensors
    return torch.tensor(x), torch.tensor(y)

n_train = 1024

train_data = sample_generator_regression(n_train)
valid_data = sample_generator_regression(64)
test_data = sample_generator_regression(128)


In [ ]:
# It is very practical to transform our data into torch dataset
train_dataset = TensorDataset(*train_data)
valid_dataset = TensorDataset(*valid_data)
test_dataset = TensorDataset(*test_data)

## Task 1: Linear layer NN.

In [ ]:
class LinearNet(nn.Module):
    ### define you model here

network = LinearNet()


Train the model

In [ ]:
# Define a poutyne model with optimizer, loss and metrics
# model = Model (...)

# optimization paramters
optimization_kwargs = {}
optimization_kwargs["batch_size"] = 8
optimization_kwargs["epochs"] = 15

# train the model
# note that the function will load the model with best validation score at the end
history = model.fit_dataset(train_dataset, valid_dataset=valid_dataset, **optimization_kwargs) 

Evaluate the model

What is the MSE on the test set?

In [ ]:
model.evaluate_dataset(test_dataset)

Have a look a the predicted function over the 2D space

In [ ]:
# this cell will produce a 2D plot of your model
x = np.arange(-lim, lim, 0.05)
y = np.arange(-lim, lim, 0.05)
xx, yy = np.meshgrid(x, y, sparse=False)
Z1 = func(xx,yy)

X = torch.tensor(np.array([xx.reshape(-1), yy.reshape(-1)]).T.astype(np.float32))
Z2 = model.predict(X).reshape(xx.shape)

vmin = np.min(Z1)
vmax = np.max(Z1)
plt.figure(figsize=(10, 5))
plt.subplot(1,2,1)
plt.imshow(Z1, vmin=vmin, vmax=vmax, extent=[-lim,lim,-lim,lim])
plt.colorbar()
plt.axis('scaled')
plt.title("Ground truth")

plt.subplot(1,2,2)
plt.imshow(Z2, vmin=vmin, vmax=vmax, extent=[-lim,lim,-lim,lim])
plt.colorbar()
plt.axis('scaled')
plt.title("Neural network");

## Task 2: Architecture adapted to the dataset
Create a new model that take into account of the shape of $f$.

Evaluate the new model and compare with the previous result.

## Task 2b: Learn the constant a, b
Make the variable a and b learnable in the model.

Did you learn the correct constant a and b?